<a href="https://colab.research.google.com/github/paulodowd/EMATM0054_53_23-24/blob/main/Labsheets/Supp/SL5_RetrieveSerialData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supplementary Labsheet 5: Retrieve Serial Data before Reset

This labsheet will help you to retrieve data from your 3Pi+ robot **after** the end of your experiment.  This should let you run your experiment without the USB cable plugged in.

The 3Pi+ has a known issue:
- Unplugging the USB cable during the robot operation can cause irratic behaviour, and/or a reset of your robot.
- Plugging in the USB cable during the robot operation can cause a reset.

This labsheet provides example code of a method to retrieve data via the Serial connection (USB).  You will need to adapt this code to your own needs, following the convention as laid out.

**Note:** Your robot will still reset when you finally close the Serial monitor.

When using this code and method, it is a good idea to reset your computer.  This will help to ensure that you do not have any existing problems with your USB ports and the Arduino IDE.

You may also want to investigate the EEPROM memory in another Supplementary Labsheet, which will allow your robot to store data even when it has been powered off or reset.  Using EEPROM and the push buttons, you can select the mode of your between running an experiment or reporting results.

<hr><br><br><br><br>

## Example Code

Using the following example code:
1. Upload the below example code.
2. Unplug your 3Pi+ from USB.
3. Activate your 3Pi+ by the motor power button.
4. Your 3Pi+ should spin for 3 seconds, then stop.
5. Plug in the USB cable.
6. Open the Serial Monitor to review reported results.
  - You can copy and paste from the Serial Monitor.
7. Closing the Serial Monitor will reset your robot.

The conventions followed in this code are:
- To include the `#define USB_ALIVE ...` code at the top of your program.
- To preface a Serial.print / Serial.println command with `if( SERIAL_ALIVE ) { ... } `.
- To keep Serial print data short (e.g., not long strings).
- To add a short `delay()` between Serial print operations.
- To design your program so the robot transitions into a permanent state of reporting the results at the end.


```c

/*
 * A known problem is that removing the USB cable during
 * the robot operation can cause very irratic behaviour.
 * Plugging in a USB cable can also reset your robot.
 * The root cause is not known (memory error?), but the
 * following example provides a solution using this
 * SERIAL_ACTIVE macro to check if the USB cable is
 * plugged in.
 */
#include <USBCore.h>    // To fix serial print behaviour bug.
u8 USB_SendSpace(u8 ep);
#define SERIAL_ACTIVE (USB_SendSpace(CDC_TX) >= 50)


// Motor Pin definitions.
#define L_PWM 10
#define L_DIR 16
#define R_PWM 9
#define R_DIR 15

// Buzzer
#define BUZZ_PIN 6


// We will store our results as a global
// variable. Just as an example, let's store
// 100 x,y, theta values.
#define MAX_RESULTS 100
float results[ MAX_RESULTS ][3];
int results_index;

// State machine
#define STATE_RUNNING_EXPERIMENT  0
#define STATE_FINISHED_EXPERIMENT 1
int state;

// Time stamp (ts) to later calculate whether
// an elapsed time is more than
//  RECORD_RESULTS_MS (100ms here)
unsigned long record_results_ts;
#define RECORD_RESULTS_MS 100

// Time stamp to track whether the experiment
// duration has elapsed.
unsigned long experiment_start_ts;
#define EXPERIMENT_END_MS 5000 // 5 seconds?

// put your setup code here, to run once:
void setup() {

  pinMode( BUZZ_PIN, OUTPUT );
  pinMode( L_PWM, OUTPUT );
  pinMode( R_PWM, OUTPUT );
  pinMode( L_DIR, OUTPUT );
  pinMode( R_DIR, OUTPUT );

  // Beep so we know when robot has reset
  analogWrite( BUZZ_PIN, 120 );
  delay( 200 );
  analogWrite( BUZZ_PIN, 0 );

  // Open serial port.
  Serial.begin(9600);
  delay(1000);
  if( SERIAL_ACTIVE )Serial.println("***RESET***");
  

  // Start robot in running experiment state.
  state = STATE_RUNNING_EXPERIMENT;

  // Record start timestamp
  experiment_start_ts millis();
  record_results_ts = millis();
  
  // Set to start recording results into
  // row 0 of our results array
  results_index = 0;
}


void loop() {

  if( state == STATE_RUNNING_EXPERIMENT ) {

    // Robot active, spinning on the spot.
    digitalWrite( L_DIR, LOW );
    digitalWrite( R_DIR, HIGH );
    analogWrite( L_PWM, 60 );
    analogWrite( R_PWM, 60 );

    unsigned long elapsed_time;

    // Check if "RESULTS_RECORD_MS" (100ms) has
    // has passed, if so, record result
    elapsed_time = millis() - record_results_ts;
    if( elapsed_time > RESULTS_RECORD_MS ) {

        // Move time stamp forwards for next
        // iteration.
        record_results_ts = millis();

        // Check we have not saved beyond our
        // max results (row) value
        if( results_index < MAX_RESULTS ) {
          results[ results_index ][0] = 0; // save x (from kinematics?)
          results[ results_index ][1] = 0; // save y (from kinematics?)
          results[ results_index ][2] = 0; // save theta (from kinematics?)

          // Increment row for next time.
          results_index++;

        } else {
          // If RESULTS_MAX has been reached, it would
          // also make sense to update state to
          // STATE_FINISHED_EXPERIMENT here.
        }

    }


    // Change to end of experiment?
    elapsed_time = millis() - experiment_start_ts;
    if( elapsed_time > EXPERIMENT_END_MS ) {
      
      // Transition to other state.
      state = STATE_FINISHED_EXPERIMENT;
    }
    

  } else if ( state == STATE_FINISHED_EXPERIMENT) {

    // Once this state has been reached, the robot
    // will remain in this state.  This state just
    // prints out all the results, then delays 3
    // seconds so you can copy-paste from serial
    // monitor.

    // Stop robot
    analogWrite( L_PWM, 0 );
    analogWrite( R_PWM, 0 );

    // Print results - uses USB fix.
    reportResultsOverSerial();

    // Delay, we don't need to do this
    // excessively fast.
    delay(3000);
      

  }

}

/*
 * Checks if the Serial connection is alive
 * before it attempts to transmit.
 * This seems to be more stable when:
 * - short amounts of data is transmitted
 * - there is small delay between transmission
 * You'll have to check that your data is
 * reported without any missing elements.
 */
void reportResultsOverSerial() {

  // Print millis for debug so we can
  // validate this is working in real
  // time, and not glitched somehow
  if( SERIAL_ACTIVE ) Serial.print( "Time(ms): " );
  if( SERIAL_ACTIVE ) Serial.println( millis() );
  delay(1);
  

  // Loop through array to print all
  // results collected
  int row = 0;
  int col = 0;  
  for( row = 0; row < MAX_RESULTS; row++ ) {

    // Print which row this is
    if( SERIAL_ACTIVE ) Serial.print( row );
        
    delay(1);
    
    
    if( SERIAL_ACTIVE ) Serial.print( "," );
    
    delay(1);
    

    // print x y theta
    for( col = 0; col < 3; col++ ) { // 3: x,y,theta

      // Comma seperated values, to 2 decimal places
      if( SERIAL_ACTIVE ) Serial.print( results[row][col], 2 );
      
      delay(1);
      
      if( SERIAL_ACTIVE ) Serial.print( "," );
      
      delay(1);
      
    }

    // Printed "x,y,theta", now needs a new line.
    if( SERIAL_ACTIVE ) Serial.print( "\n" ); // new row
  }

  if( SERIAL_ACTIVE ) Serial.println( "---End of Results ---\n\n" );
  
}

```

Expected output on Serial Monitor:

```c
Time(ms): 21374
0, 0.00, 0.00, 0.00,
1, 0.00, 0.00, 0.00,
2, 0.00, 0.00, 0.00,
3, 0.00, 0.00, 0.00,
4, 0.00, 0.00, 0.00,
5, 0.00, 0.00, 0.00,
(etc)
---End of Results ---

```